In [ ]:
cfg_id_column = 0
cfg_title_column = 2
cfg_depend_column = 6
cfg_first_valid_row = 1
cfg_project_prefix_cfid = 21
cfg_rmtracker_id = 3
cfg_rmtitle_cfid = 16


In [ ]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

In [ ]:
redmine = Redmine(rm_server_url,key=rm_key_txt)
projects = redmine.project.all()

In [ ]:
print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

my_project = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

In [ ]:
file_path = "input_dsm.ods"
data = get_data(file_path)['_RqDep']
tasks = {}

In [ ]:

i = 0
tasks = {}
for row in data:
    # Let's skip the first row
    if (i >= cfg_first_valid_row):
        thisTsk = {}
        if (len(row) > cfg_title_column) and (len(row[cfg_title_column]) > 0):
            thisTsk['rmid'] = 0
            thisTsk['rmident'] = ''
            thisTsk['row'] = i
            thisTsk['ident'] = row[cfg_id_column]
            thisTsk['title'] = row[cfg_title_column]
            if (len(row) > cfg_depend_column):
                thisTsk['dependences'] = row[cfg_depend_column].split(' ')
            else:
                thisTsk['dependences'] = []
                
            #print(thisTsk)
            tasks[thisTsk['ident']] = thisTsk
        
    i += 1
    


In [ ]:
print(tasks.keys())

Vamos a buscar las mismas tareas al Redmine, si es posible encontrarlas

In [ ]:
# Buscamos el prefijo del proyecto
prjPrefix = my_project.custom_fields.get(cfg_project_prefix_cfid).value
lenPrjPrefix = len(prjPrefix)
# Recorreremos los identificadores de las tareas cargadas del ODS para ver si alguno se corresponde con un identificador válido de proyecto
for tskey in tasks.keys():
    thisRmTsk = None
    if (len(tskey)>lenPrjPrefix+1):
        tskPrefix = tskey[0:lenPrjPrefix]
        if (tskPrefix == prjPrefix) and (tskey[lenPrjPrefix] == '-'):
            print("Identificador rm:",tskey)
            thisRmTskList = myproject.issues.filter(status_id='*',subject=tskey)
            if (len(thisRmTskList)==1):
                # hemos encontrado una tarea de Redmine que cuadra
                thisRmTsk = thisRmTskList[0]
                tasks[tskey]['rmident'] = tskey
                tasks[tskey]['rmid'] = thisRmTsk.id
                tasks[tskey]['rmtsk'] = thisRmTsk
                
    if (thisRmTsk == None):
        print("Tarea nueva",tskey)
        thisRmTsk = redmine.issue.create(project_id = my_project.id,
            tracker_id = cfg_rmtracker_id,
            description = tasks[tskey]['title'],
            custom_fields=[
                {'id': cfg_rmtitle_cfid,'value': tasks[tskey]['title']}
            ]
        )
        tasks[tskey]['rmident'] = thisRmTsk.subject
        tasks[tskey]['rmid'] = thisRmTsk.id
        tasks[tskey]['rmtsk'] = thisRmTsk


In [ ]:
for tskey in tasks.keys():
    print("id",tskey,"rmident",tasks[tskey]['rmident'])

In [ ]:
# Recorreremos de nuevo los identificadores de las tareas cargadas del ODS para crear/modificar sus relaciones
for tskey in tasks.keys():
    odsDependences = tasks[tskey]['dependences']
    print("id",tskey,odsDependences)
    rmDependences = []
    for dpd in odsDependences:
        thisRmId = tasks[dpd]['rmid']
        print(thisRmId)
        rmDependences.append(thisRmId)
        
    tasks[tskey]['rmdep'] = rmDependences
    

In [ ]:
# Ahora buscamos las dependencias ya existentes en el Redmine y las restamos de las relaciones a añadir
for tskey in tasks.keys():
    print("id",tskey,tasks[tskey]['dependences'],tasks[tskey]['rmdep'])
    thisTskRmId = tasks[tskey]['rmid']
    thisTskRelations = tasks[tskey]['rmtsk'].relations
    thisTskInputRelations = list(filter(lambda x: x.issue_to_id == thisTskRmId and relation_type=='precedes', thisTskRelations))
    print("Preexistent:", thisTskInputRelations)
    for rel in thisTskInputRelations:
        if rel.issue_to_id in tasks[tskey]['rmdep']:
            # La relación está a punto de ser añadida, cancelamos su creación
            tasks[tskey]['rmdep'].remove(rel.issue_to_id)
            print("En",tskey,"Borro",rel.issue_to_id,"de",tasks[tskey]['rmdep'])
        
        else:
            # La relación ya no existe, hay que borrarla
            redmine.issue_relation.delete(rel.id)
            
    # Ahora añadimos las relaciones restantes
    for relId in tasks[tskey]['rmdep']:
        rel = redmine.issue_relation.create(
            issue_id=relId,
            issue_to_id=thisTskRmId,
            relation_type='precedes',
            delay=0
        )

            
